In [8]:
!pip install git+https://github.com/jpata/gym-so100.git@integration
!pip install imageio torch

  Cloning https://github.com/jpata/gym-so100.git (to revision integration) to /tmp/pip-req-build-o766efg9
  Running command git clone --filter=blob:none --quiet https://github.com/jpata/gym-so100.git /tmp/pip-req-build-o766efg9
  Running command git checkout -b integration --track origin/integration
  Switched to a new branch 'integration'
  Branch 'integration' set up to track remote branch 'integration' from 'origin'.
  Resolved https://github.com/jpata/gym-so100.git to commit d3dd545d9bfec4639a6e4c631dc62dee138d35ba
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached nvid

In [1]:
import imageio
import gymnasium as gym
import numpy as np
import gym_so100
import numpy
import torch

In [2]:
env = gym.make("gym_so100/PushCube-v0", render_mode="human")
observation, info = env.reset()
frames = []

In [19]:
from lerobot.common.policies.act.modeling_act import ACTPolicy
pretrained_path = "/home/absin/Documents/git/robotics/act_algorithm/trained_model/act_aloha_sim_transfer_cube_human"
policy = ACTPolicy.from_pretrained(pretrained_path)
policy.reset()
numpy_observation, info = env.reset(seed=42)
rewards = []
frames = []
# Render frame of the initial state
frames.append(env.render())
step = 0
done = False
device = 'cpu'
while not done:
    # Prepare observation for the policy running in Pytorch
    state = torch.from_numpy(numpy_observation["agent_pos"])
    image = torch.from_numpy(numpy_observation["image_top"])
    state = torch.zeros(14)
    state[:numpy_observation["agent_pos"].shape[0]] = torch.from_numpy(numpy_observation["agent_pos"])
    # Convert to float32 with image from channel first in [0,255]
    # to channel last in [0,1]
    state = state.to(torch.float32)
    image = image.to(torch.float32) / 255
    image = image.permute(2, 0, 1)

    # Send data tensors from CPU to GPU
    state = state.to(device, non_blocking=True)
    image = image.to(device, non_blocking=True)

    # Add extra (empty) batch dimension, required to forward the policy
    state = state.unsqueeze(0)
    image = image.unsqueeze(0)

    # Create the policy input dictionary
    observation = {
        "observation.state": state,
        "observation.images.top": image,
    }

    # Predict the next action with respect to the current observation
    with torch.inference_mode():
        action = policy.select_action(observation)

    # Prepare the action for the environment
    numpy_action = action.squeeze(0).to("cpu").numpy()

    # Step through the environment and receive a new observation
    numpy_observation, reward, terminated, truncated, info = env.step(numpy_action[:6])
    print(f"{step=} {reward=} {terminated=}")

    # Keep track of all the rewards and frames
    rewards.append(reward)
    frames.append(env.render())

    # The rollout is considered done when the success state is reach (i.e. terminated is True),
    # or the maximum number of iterations is reached (i.e. truncated is True)
    done = terminated | truncated | done
    step += 1

if terminated:
    print("Success!")
else:
    print("Failure!")

imageio.mimsave('5_so100.mp4', numpy.stack(frames), fps=25)
print(f"Video of the evaluation is available in 5_so100.mp4.")

Loading weights from local directory
step=0 reward=-5.32459926506402 terminated=False
step=1 reward=-5.372093170162899 terminated=False
step=2 reward=-5.540144321841882 terminated=False
step=3 reward=-5.698087689052722 terminated=False
step=4 reward=-6.884907322121309 terminated=False
step=5 reward=-6.881013239445432 terminated=False
step=6 reward=-6.857959521922565 terminated=False
step=7 reward=-6.844647497256892 terminated=False
step=8 reward=-5.818788504558431 terminated=False
step=9 reward=-5.753434978289583 terminated=False
step=10 reward=-5.734157096465994 terminated=False
step=11 reward=-5.795307432858733 terminated=False
step=12 reward=-5.977746005789837 terminated=False
step=13 reward=-6.187914176257796 terminated=False
step=14 reward=-6.134316480585964 terminated=False
step=15 reward=-6.112694074209591 terminated=False
step=16 reward=-6.093202495994923 terminated=False
step=17 reward=-6.0521035037763316 terminated=False
step=18 reward=-6.029147152453978 terminated=False
step

ValueError: Image data must be a sequence of ndimages.

In [23]:
env.render()

{'agent_pos': array([-2.2       , -3.1416435 ,  3.141698  , -2.0000236 , -3.141577  ,
        -0.20000166], dtype=float32),
 'agent_vel': array([-8.6965530e-14, -1.0769277e-13, -1.0667864e-13, -1.0825468e-13,
         3.2166736e-09, -5.4392504e-15], dtype=float32),
 'target_pos': array([ 0.0273956 , -0.20244487,  0.01      ], dtype=float32),
 'image_front': array([[[39, 66, 92],
         [39, 66, 92],
         [39, 66, 92],
         ...,
         [39, 66, 92],
         [39, 66, 92],
         [39, 66, 92]],
 
        [[39, 66, 92],
         [39, 66, 92],
         [39, 66, 92],
         ...,
         [39, 66, 92],
         [39, 66, 92],
         [39, 66, 92]],
 
        [[39, 65, 92],
         [39, 65, 92],
         [39, 65, 92],
         ...,
         [39, 65, 92],
         [39, 65, 92],
         [39, 65, 92]],
 
        ...,
 
        [[29, 59, 87],
         [29, 59, 87],
         [29, 59, 87],
         ...,
         [29, 59, 88],
         [29, 59, 88],
         [29, 59, 88]],
 
      

In [11]:
padded_agent_pos = torch.zeros(14)
padded_agent_pos[:numpy_observation["agent_pos"].shape[0]] = torch.from_numpy(numpy_observation["agent_pos"])


In [13]:
padded_agent_pos.shape

torch.Size([14])

In [7]:
!pip uninstall torch

Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Would remove:
    /home/absin/Documents/git/robotics/gym-so100/venv/bin/torchfrtrace
    /home/absin/Documents/git/robotics/gym-so100/venv/bin/torchrun
    /home/absin/Documents/git/robotics/gym-so100/venv/lib/python3.10/site-packages/functorch/*
    /home/absin/Documents/git/robotics/gym-so100/venv/lib/python3.10/site-packages/torch-2.6.0.dist-info/*
    /home/absin/Documents/git/robotics/gym-so100/venv/lib/python3.10/site-packages/torch/*
    /home/absin/Documents/git/robotics/gym-so100/venv/lib/python3.10/site-packages/torchgen/*
^Coceed (Y/n)? 
ERROR: Operation cancelled by user
